In [2]:
from flipside import Flipside
import numpy as np
import pandas as pd 
import plotly.graph_objects as go
import pmdarima as pm

api_key = '07473093-5657-4739-b1bb-98aeeb2f26c1'
flipside = Flipside(api_key, "https://api-v2.flipsidecrypto.xyz")


ModuleNotFoundError: No module named 'plotly'

#### Query 1: Historic Price Data

In [44]:
# Calculate date range
end_date = datetime.now().date()
start_date = end_date - timedelta(days=50000)  # 2 years ago

# SQL query
sql = f"""
SELECT
    date_trunc('day', hour) as date,
    MIN(price) as low,
    MAX(price) as high,
    AVG(CASE WHEN EXTRACT(HOUR FROM hour) = 0 THEN price END) as open,
    AVG(CASE WHEN EXTRACT(HOUR FROM hour) = 23 THEN price END) as close
FROM BITCOIN.price.ez_prices_hourly
WHERE symbol = 'BTC'
    AND EXTRACT(YEAR FROM hour) BETWEEN 2020 AND 2024
GROUP BY date_trunc('day', hour)
ORDER BY date_trunc('day', hour);
"""

query_result_set = flipside.query(sql)

#### Query 2: Volume

In [43]:
volume_sql = """
SELECT
    DATE_TRUNC('day', BLOCK_TIMESTAMP) AS date,
    SUM(OUTPUT_VALUE) AS bitcoin_volume
FROM
    bitcoin.core.fact_transactions -- Replace with your actual table name
WHERE 1=1
    -- COINBASE = 'Bitcoin' -- Assuming COINBASE column indicates Bitcoin transactions
GROUP BY
    DATE_TRUNC('day', BLOCK_TIMESTAMP)
ORDER BY
    DATE_TRUNC('day', BLOCK_TIMESTAMP);
"""

Volume = flipside.query(volume_sql)
Volume = auto_paginate_result(query_result_set=Volume)
Volume = pd.DataFrame(Volume)
Volume

,date,bitcoin_volume,__row_index
0,2009-01-03T00:00:00.000Z,50.000000,0
1,2009-01-09T00:00:00.000Z,700.000000,1
2,2009-01-10T00:00:00.000Z,3050.000000,2
3,2009-01-11T00:00:00.000Z,4650.000000,3
4,2009-01-12T00:00:00.000Z,4879.000000,4
...,...,...,...
5651,2024-06-29T00:00:00.000Z,453278.115420,5651
5652,2024-06-30T00:00:00.000Z,373215.581249,5652
5653,2024-07-01T00:00:00.000Z,664824.909421,5653
5654,2024-07-02T00:00:00.000Z,634181.730253,5654


#### Function to convert into DF format

In [45]:
def auto_paginate_result(query_result_set, page_size=10000):
    """
    This function auto-paginates a query result to get all the data. It assumes 10,000 rows per page.
    In case of an error, reduce the page size. Uses numpy.
    """
    num_rows = query_result_set.page.totalRows
    page_count = np.ceil(num_rows / page_size).astype(int)
    all_rows = []
    current_page = 1
    while current_page <= page_count:
        results = flipside.get_query_results(
            query_result_set.query_id,
            page_number=current_page,
            page_size=page_size
        )

        if results.records:
            all_rows.extend(results.records)  # Use extend() to add list elements

        current_page += 1  # Increment the current page number

    return all_rows  # Return all_rows in JSON format

results = auto_paginate_result(query_result_set=query_result_set)
results = pd.DataFrame(results)
results

,date,low,high,open,close,__row_index
0,2020-01-01T00:00:00.000Z,7176.662158,7256.235116,7200.685899,7223.949583,0
1,2020-01-02T00:00:00.000Z,6961.119459,7222.235506,7207.135177,6986.753868,1
2,2020-01-03T00:00:00.000Z,6891.250830,7364.726154,6974.922052,7300.537592,2
3,2020-01-04T00:00:00.000Z,7301.334593,7375.768309,7303.722429,7362.912925,3
4,2020-01-05T00:00:00.000Z,7347.325787,7483.968613,7371.236016,7376.189807,4
...,...,...,...,...,...,...
1641,2024-06-29T00:00:00.000Z,60238.000000,61026.000000,60238.000000,60956.000000,1641
1642,2024-06-30T00:00:00.000Z,60645.000000,61984.000000,60910.000000,61820.000000,1642
1643,2024-07-01T00:00:00.000Z,62547.000000,63688.000000,62695.000000,62905.000000,1643
1644,2024-07-02T00:00:00.000Z,61852.000000,63067.000000,62902.000000,61958.000000,1644


In [64]:
results.date = pd.to_datetime(results.date)
Volume.date = pd.to_datetime(Volume.date)

btc_data = pd.merge(results, Volume, on='date', how='left')

from plotly.subplots import make_subplots

# Assuming btc_data is your existing dataframe
# Add 30-day moving averages for price and volume
btc_data['price_ma30'] = btc_data['close'].rolling(window=30).mean()
btc_data['volume_ma30'] = btc_data['bitcoin_volume'].rolling(window=30).mean()

# Create a figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Color scheme
colors = {
    'background': '#F5F5DC',  # Beige
    'price': '#8B4513',       # Saddle Brown
    'volume': '#D2691E',      # Chocolate
    'ma_price': '#A52A2A',    # Brown
    'ma_volume': '#CD853F'    # Peru
}

# Add traces
#fig.add_trace(go.Scatter(x=btc_data.date, y=btc_data.close, mode='lines', name='Close Price',
                         #line=dict(color=colors['price'], width=1.5)), secondary_y=False)
fig.add_trace(go.Scatter(x=btc_data.date, y=btc_data.price_ma30, mode='lines', name='30-day MA (Price)',
                         line=dict(color=colors['ma_price'], width=2)), secondary_y=False)
#fig.add_trace(go.Scatter(x=btc_data.date, y=btc_data.bitcoin_volume, mode='lines', name='Bitcoin Volume',
                         #line=dict(color=colors['volume'], width=1.5)), secondary_y=True)
fig.add_trace(go.Scatter(x=btc_data.date, y=btc_data.volume_ma30, mode='lines', name='30-day MA (Volume)',
                         line=dict(color=colors['ma_volume'], width=2, dash='dash')), secondary_y=True)

# Customize the layout
fig.update_layout(
    title={
        'text': 'Bitcoin Price and Volume Evolution',
        'font': {'size': 24, 'color': '#4A4A4A', 'family': 'Garamond'},
        'x': 0.5,
        'y': 0.95
    },
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    xaxis=dict(
        title='Date',
        titlefont=dict(size=16, color='#4A4A4A'),
        showgrid=True,
        gridcolor='rgba(169,169,169,0.3)',
        tickfont=dict(size=12, color='#4A4A4A')
    ),
    yaxis=dict(
        title='Price (USD)',
        titlefont=dict(size=16, color=colors['price']),
        tickfont=dict(size=12, color=colors['price']),
        showgrid=True,
        gridcolor='rgba(169,169,169,0.3)',
    ),
    yaxis2=dict(
        title='Volume',
        titlefont=dict(size=16, color=colors['volume']),
        tickfont=dict(size=12, color=colors['volume']),
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='#4A4A4A',
        borderwidth=1,
        font=dict(size=12, color='#4A4A4A')
    ),
    width=2000,
    height=700,
    margin=dict(l=80, r=80, t=100, b=80),
)

# Add a watermark
fig.add_annotation(
    text="CamelQuant",
    x=0.5,
    y=0.5,
    xref="paper",
    yref="paper",
    showarrow=False,
    font=dict(size=100, color="rgba(169,169,169,0.4)", family="Brush Script MT"),
    textangle=-30
)

# Update axes
fig.update_xaxes(showline=True, linewidth=2, linecolor='#4A4A4A', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='#4A4A4A', mirror=True)

# Show the plot
fig.show()

https://coinmarketcap.com/currencies/bitcoin/historical-data/

In [134]:
btc_all = pd.read_csv('bitcoin_all.csv', delimiter=';')
gtrends = pd.read_csv('gtrends.csv', header=1)
reddit = pd.read_csv('reddit_sentiment.csv')
gtrends.Day = pd.to_datetime(gtrends.Day, utc=True)
reddit.date_posted = pd.to_datetime(reddit.date_posted, utc=True)
columns_to_convert = ['timeOpen', 'timeClose', 'timeHigh', 'timeLow', 'timestamp']
btc_all[columns_to_convert] = btc_all[columns_to_convert].apply(lambda col: pd.to_datetime(col, utc=True))
btc_all = btc_all.sort_values('timestamp').reset_index(drop=True)
btc_all = btc_all.drop('name', axis=1)
btc_all = pd.merge(btc_all, gtrends, left_on='timeOpen', right_on='Day', how='left')
btc_all = pd.merge(btc_all, reddit, left_on='timeOpen', right_on='date_posted', how='left')
btc_all = btc_all[['timeOpen', 'open', 'high', 'low', 'close', 'volume', 'marketCap', 'bitcoin: (Worldwide)', 'weighted_sen']]
btc_all

,timeOpen,open,high,low,close,volume,marketCap,bitcoin: (Worldwide),weighted_sen
0,2023-07-05 00:00:00+00:00,30778.725363,30877.329251,30225.612699,30514.166393,1.248162e+10,5.926152e+11,NaN,NaN
1,2023-07-06 00:00:00+00:00,30507.151111,31460.053613,29892.226032,29909.337761,2.112922e+10,5.808949e+11,NaN,NaN
2,2023-07-07 00:00:00+00:00,29907.998902,30434.643694,29777.284410,30342.264752,1.338477e+10,5.893367e+11,NaN,NaN
3,2023-07-08 00:00:00+00:00,30346.921653,30374.436525,30080.159610,30292.541173,7.509379e+09,5.884044e+11,NaN,NaN
4,2023-07-09 00:00:00+00:00,30291.610530,30427.590660,30085.592193,30171.233572,7.903328e+09,5.860798e+11,NaN,NaN
...,...,...,...,...,...,...,...,...,...
359,2024-06-28 00:00:00+00:00,61612.805785,62126.096568,59985.404260,60320.135526,2.495287e+10,1.189406e+12,24.0,0.005609
360,2024-06-29 00:00:00+00:00,60319.873202,61097.621824,60300.965102,60887.380835,1.265290e+10,1.200610e+12,21.0,-0.199729
361,2024-06-30 00:00:00+00:00,60888.445562,62892.827199,60632.950439,62678.292079,1.733323e+10,1.235945e+12,21.0,0.014421
362,2024-07-01 00:00:00+00:00,62673.606339,63777.227542,62495.510104,62851.980068,2.546838e+10,1.239386e+12,26.0,0.279215


In [145]:
from sklearn.preprocessing import MinMaxScaler
btc_ai = btc_all.copy()


# Calculate Volatility
def calculate_volatility(prices_df):
    prices_df['return'] = prices_df['close'].pct_change()
    volatility_df = prices_df['return'].rolling(window=30).std() * np.sqrt(30)  # Monthly volatility
    scaler = MinMaxScaler((0, 100))
    prices_df['volatility_score'] = scaler.fit_transform(volatility_df.values.reshape(-1, 1))
    return prices_df

# Normalize
def normalize(volume_df, column):
    scaler = MinMaxScaler((0, 100))
    volume_df[f'{column}_normal'] = scaler.fit_transform(volume_df[[column]])
    return volume_df


btc_ai = calculate_volatility(btc_ai)
btc_ai = normalize(btc_ai, 'volume')
btc_ai = normalize(btc_ai, 'bitcoin: (Worldwide)')
btc_ai = normalize(btc_ai, 'weighted_sen')
btc_ai

,timeOpen,open,high,low,close,volume,marketCap,bitcoin: (Worldwide),weighted_sen,return,volatility_score,volume_normal,bitcoin: (Worldwide)_normal,weighted_sen_normal
0,2023-07-05 00:00:00+00:00,30778.725363,30877.329251,30225.612699,30514.166393,1.248162e+10,5.926152e+11,NaN,NaN,NaN,NaN,7.335919,NaN,NaN
1,2023-07-06 00:00:00+00:00,30507.151111,31460.053613,29892.226032,29909.337761,2.112922e+10,5.808949e+11,NaN,NaN,-0.019821,NaN,16.207818,NaN,NaN
2,2023-07-07 00:00:00+00:00,29907.998902,30434.643694,29777.284410,30342.264752,1.338477e+10,5.893367e+11,NaN,NaN,0.014475,NaN,8.262492,NaN,NaN
3,2023-07-08 00:00:00+00:00,30346.921653,30374.436525,30080.159610,30292.541173,7.509379e+09,5.884044e+11,NaN,NaN,-0.001639,NaN,2.234704,NaN,NaN
4,2023-07-09 00:00:00+00:00,30291.610530,30427.590660,30085.592193,30171.233572,7.903328e+09,5.860798e+11,NaN,NaN,-0.004005,NaN,2.638872,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2024-06-28 00:00:00+00:00,61612.805785,62126.096568,59985.404260,60320.135526,2.495287e+10,1.189406e+12,24.0,0.005609,-0.020853,20.579291,20.130643,3.797468,28.283305
360,2024-06-29 00:00:00+00:00,60319.873202,61097.621824,60300.965102,60887.380835,1.265290e+10,1.200610e+12,21.0,-0.199729,0.009404,20.387104,7.511642,0.000000,13.466665
361,2024-06-30 00:00:00+00:00,60888.445562,62892.827199,60632.950439,62678.292079,1.733323e+10,1.235945e+12,21.0,0.014421,0.029414,23.194596,12.313364,0.000000,28.919151
362,2024-07-01 00:00:00+00:00,62673.606339,63777.227542,62495.510104,62851.980068,2.546838e+10,1.239386e+12,26.0,0.279215,0.002771,23.181824,20.659527,6.329114,48.026066


In [1]:
camel_fng = btc_ai[['timeOpen', 'volatility_score', 'volume_normal', 'bitcoin: (Worldwide)_normal', 'weighted_sen_normal']]
camel_fng = camel_fng.dropna()
camel_fng['FnG'] = camel_fng.volatility_score*0.3 + camel_fng.volume_normal*0.3 + camel_fng['bitcoin: (Worldwide)_normal']*0.2 + camel_fng.weighted_sen_normal*0.2
camel_fng = camel_fng[camel_fng['timeOpen'].dt.month >= 2]

NameError: name 'btc_ai' is not defined

In [168]:
# Create the plot with Plotly
fig = go.Figure()

# Add the Fear and Greed Index line
fig.add_trace(go.Scatter(x=camel_fng['timeOpen'], y=camel_fng['FnG'], mode='lines', name='FnG Index',
                         line=dict(color='black')))

# Add horizontal lines for 70 and 30
fig.add_shape(type='line', x0=camel_fng['timeOpen'].min(), x1=camel_fng['timeOpen'].max(), y0=50, y1=50,
              line=dict(color='green', dash='dot'), name='70 Line')
fig.add_shape(type='line', x0=camel_fng['timeOpen'].min(), x1=camel_fng['timeOpen'].max(), y0=30, y1=30,
              line=dict(color='red', dash='dot'), name='30 Line')

# Update layout for a desert theme
fig.update_layout(
    title='Fear and Greed Index Over Time',
    xaxis_title='Date',
    yaxis_title='Fear and Greed Index',
    font=dict(family='Garamond, monospace', size=18, color='black'),
    paper_bgcolor='beige',
    plot_bgcolor='white',
    title_font=dict(size=24, color='brown', family='Garamond'),
    annotations=[
        dict(
            text='CamelQuant',
            xref='paper', yref='paper',
            x=0.5, y=0.5,
            xanchor='center', yanchor='middle',
            font=dict(family='Arial', size=50, color='rgba(165, 42, 42, 0.2)'),
            showarrow=False
        )
    ]
)

fig.show()